- [recsys 2015 challenge](https://recsys.yoochoose.net/challenge.html) dataset
- (참고) 7z 확장자로 압축되어 있음. 다운로드 및 압축푸는 과정은 생략함.

- ![aladin](./asset/시크릿모드.png)

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [54]:
PATH_TO_ORIGINAL_DATA = 'D:\\data\\yoochoose-data\\'
PATH_TO_PROCESSED_DATA = 'D:\\data\\yoochoose-data\\'

In [63]:
data = pd.read_csv(PATH_TO_ORIGINAL_DATA + 'yoochoose-clicks.dat', sep=',', header=None, usecols=[0,1,2],
                   parse_dates=[1], 
                   dtype={0:np.int32, 2:np.int32}, nrows=100000)
data.columns = ['SessionId', 'Time', 'ItemId']

In [64]:
session_lengths = data.groupby('SessionId').size()
data = data[np.in1d(data.SessionId, session_lengths[session_lengths>1].index)]

In [65]:
item_supports = data.groupby('ItemId').size()
data = data[np.in1d(data.ItemId, item_supports[item_supports>=5].index)]

In [66]:
session_lengths = data.groupby('SessionId').size()
data = data[np.in1d(data.SessionId, session_lengths[session_lengths>=2].index)]

In [82]:
tmax = data['Time'].max()
session_max_times = data.groupby('SessionId')['Time'].max()
session_train = session_max_times[session_max_times < tmax-dt.timedelta(1)].index
session_test = session_max_times[session_max_times >= tmax-dt.timedelta(1)].index

In [83]:
train = data[np.in1d(data.SessionId, session_train)]
test = data[np.in1d(data.SessionId, session_test)]

In [85]:
test = test[np.in1d(test.ItemId, train.ItemId)]

In [91]:
test_length = test.groupby('SessionId').size()
test = test[np.in1d(test.SessionId, test_length[test_length>=2].index)]

In [92]:
print(f'Full train set\n\tEvents: {len(train)}\n\tSessions: {train.SessionId.nunique()}\n\tItems: {train.ItemId.nunique()}')
train.to_csv(PATH_TO_PROCESSED_DATA + 'rsc15_train_full.txt', sep='\t', index=False)

Full train set
	Events: 70278
	Sessions: 17794
	Items: 2933


In [93]:
print(f'Test set\n\tEvents: {len(test)}\n\tSessions: {test.SessionId.nunique()}\n\tItems: {test.ItemId.nunique()}')
test.to_csv(PATH_TO_PROCESSED_DATA + 'rsc15_test.txt', sep='\t', index=False)

Test set
	Events: 13568
	Sessions: 3416
	Items: 1771
